# Aditya Sawant's Version of SPN_IP_5Shot.ipynb

### Libraries Used

- tensorflow
- sklearn
- numpy
- matplotlib
- pandas
- scipy
- tensorflow_probability


## Initial setup

### Install Libraries

In [241]:
# %pip install scikit-learn numpy matplotlib scipy tensorflow_probability pandas plotly 
# tensorflow[and-cuda]==2.10 [cuDNN 8.1.1 CUDA 11.2]

### Timer Function

In [242]:
from time import time

def timeIt(func):
    """
    timeIt is a decorator function to time the execution of a function.
    
    :param func: function to be timed
    :return: wrapper function
    """
    def wrap_func(*args, **kwargs):
        t1 = time()
        result = func(*args, **kwargs)
        t2 = time()
        print(f'Function {func.__name__!r} executed in {(t2-t1):.4f}s')
        return result
    return wrap_func

### Plot Data

In [243]:
# trainingData = pd.DataFrame(columns=['Loss', 'Accuracy'])
def plotData(data, testing=False):
    
    if testing:
        accuracy1_values = [item[0] for item in data]
        accuracy2_values = [item[1] for item in data]
        loss1_values = [item[2] for item in data]
        loss2_values = [item[3] for item in data]

        # Create the plot
        plt.figure(figsize=(10, 6))
        plt.plot(loss1_values, color='red', label='Loss')
        plt.plot(accuracy1_values, color='blue', label='Accuracy')
        plt.xlabel('Epoch/Episode')
        plt.ylabel('Value')
        plt.title('Loss and Overall Accuracy 1')
        plt.legend()
        plt.grid(True)
        plt.show()
        
        plt.figure(figsize=(10, 6))
        plt.plot(loss2_values, color='red', label='Loss')
        plt.plot(accuracy2_values, color='blue', label='Accuracy')
        plt.xlabel('Epoch/Episode')
        plt.ylabel('Value')
        plt.title('Loss and Overall Accuracy 2')
        plt.legend()
        plt.grid(True)
        plt.show()
    else:
        loss_values = [item[0] for item in data]
        accuracy_values = [item[1] for item in data]

        # Create the plot
        plt.figure(figsize=(10, 6))
        plt.plot(loss_values, color='red', label='Loss')
        plt.plot(accuracy_values, color='blue', label='Accuracy')
        plt.xlabel('Epoch/Episode')
        plt.ylabel('Value')
        plt.title('Loss and Accuracy')
        plt.legend()
        plt.grid(True)
        plt.show()
    clear_output(wait=True)

### Import Libraries

In [244]:
import random
import statistics
import os
from IPython.display import clear_output
  
from operator import truediv

import tensorflow as tf
print(tf.version.GIT_VERSION, tf.version.VERSION)
print(tf.test.is_built_with_cuda())
print("Number of GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Number of Devices Available: ", len(tf.config.experimental.list_physical_devices()))
# import tensorflow_probability as tfp
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
from tensorflow.python.keras import backend as K

from tensorflow.keras import Sequential, layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.compat.v1.distributions import Bernoulli

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
from datetime import date, datetime
from tqdm.auto import tqdm

from lib.Data import Data
from lib.Stats import Stats


v2.10.0-rc3-6-g359c3cdfc5f 2.10.0
True
Number of GPUs Available:  1
Number of Devices Available:  2


## Global Variables

In [245]:

# Test each code block individually
TEST_BLOCKS: bool = False
CWD: str = os.getcwd()
VERBOSE: bool = False

SAVE_REPORT: bool = True
SAVE_MODEL: bool = False
# Data Loading and Preprocessing


# Dataset Used : Indian Pines
DATASET: str = 'IP' # IP (indian_pines) PU (pavia_university) SA (salinas) HU (houston) 
PATH_TO_DATASET: str = CWD + '\\Datasets\\'

# PCA
PCA_COMPONENTS: int = 30 # Number of components to keep after PCA reduction

# Window size for forming image cubes
WINDOW_SIZE: int = 11

# Image dimensions after forming image cubes
IMAGE_WIDTH: int
IMAGE_HEIGHT: int
IMAGE_DEPTH: int
IMAGE_CHANNEL: int 
IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_DEPTH, IMAGE_CHANNEL = 11, 11, 30, 1

# Model Parameters

N_TIMES = 1 # Number of times to run the model. Internally, the model is runs each episode N_TIMES times

# Learning Rate
LEARNING_RATE: float = 0.00001

# Temprature Scaling
TAU: float = 1.8

# C (No. of Classes) K (No. of Samples per Class) N (No. of Patches per Class)
TRAIN_C: int = 5 # Number of classes to be used for training
TRAIN_K: int = 5 # Number of patches per class to be used for support during training
TRAIN_N: int = 15 # Number of patches per class to be used for query during training

TUNE_C: int = 3 # Number of classes to be used for testing
TUNE_K: int = 1 # Number of patches per class to be used for support during testing
TUNE_N: int = 4 # Number of patches per class to be used for query during testing

TEST_C: int = 3 # Number of classes to be used for testing
TEST_K: int = 5 # Number of patches per class to be used for support during testing
TEST_N: int = 5 # Number of patches per class to be used for query during testing

# ===================================
# DO NOT REMOVE THIS.
tC = 3   # classes in a test episode 
# Don't know this yet, probably used in the model to calculate loss
MC_LOSS_WEIGHT: int = 5 
# DIRECTLY USED IN PROTOTYPICAL NETWORK CLASS IN TESTING CASE
# ===================================

# Training Epochs
TRAINING_EPOCH: int = 3  # 50

# Training Episode
TRAINING_EPISODE: int = 5 # 100

# Tunning Epochs
TUNNING_EPOCH: int = 3

# Tunning Episode
TUNNING_EPISODE: int = 5

# Testing Epochs
TESTING_EPOCH: int = 10

# Metrics to be used for evaluation
train_loss = tf.metrics.Mean(name='train_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')
tune_loss = tf.metrics.Mean(name='tune_loss')
tune_acc = tf.metrics.Mean(name='tune_accuracy')
test_loss = tf.metrics.Mean(name='test_loss')
test_acc = tf.metrics.Mean(name='test_accuracy')

trainingData = []
tunningData = []
testingData = []

run_folder =  f'{date.today()}' + '-' + f'{datetime.now().hour}_5_1' + '\\' 

checkpoint_dir = CWD + '\\saves\\' + run_folder + DATASET + '\\' + f'{TRAIN_K}_shot_way' + '\\Train'
checkpoint_prefix_train = os.path.join(checkpoint_dir, "ckpt")

checkpoint_dir1 = CWD + '\\saves\\' + run_folder + DATASET + '\\' + f'{TRAIN_K}_shot_way' + '\\Train\\Tune'
checkpoint_prefix_tune = os.path.join(checkpoint_dir1, "ckpt")

report_path = CWD + f'\\Reports\\Report_{date.today()}_{str(datetime.now()).split(".")[0].split()[1].replace(":", "-")}.txt'
model_save_path = CWD + '\\saves\\' + run_folder + DATASET + '\\' + f'{TRAIN_K}_shot_way' + '\\Train\\encoder.h5'


checkpoint = None  # To be used for loading checkpoints. Declared in the Main Block
ProtoModel = None  # Prototypical Network Object. Declared in the Main Block
model = None  # Model Object. Declared in the Main Block
optimizer = None  # Optimizer Object. Declared in the Main Block

## Model

### Model Construction


In [246]:
def createModel():
    """
    createModel() function creates the model architecture for the 3D CNN model.
    :return: model 
    
    The model architecture is as follows:
    1. Input layer
    2. 3D Convolution layer with 8 filters, kernel size (3,3,7), activation function 'relu' and padding 'same'
    3. Spatial Dropout layer with dropout rate 0.3
    4. 3D Convolution layer with 16 filters, kernel size (3,3,5), activation function 'relu' and padding 'same'
    5. Spatial Dropout layer with dropout rate 0.3
    6. 3D Convolution layer with 32 filters, kernel size (3,3,3), activation function 'relu'
    7. Reshape layer to reshape the output of 3D Convolution layer to 2D
    8. 2D Convolution layer with 64 filters, kernel size (3,3), activation function 'relu'
    9. Flatten layer to flatten the output of 2D Convolution layer
    10. Dropout layer with dropout rate 0.4
    11. Dense layer with 256 neurons and activation function 'relu'
    12. Dropout layer with dropout rate 0.4
    13. Dense layer with 128 neurons and activation function 'relu'
    14. Output layer with 128 neurons and activation function 'relu'
    
    """

    # input_layer = layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH, IMAGE_CHANNEL))
    
    # output_layer_1_conv = layers.Conv3D(filters=8, kernel_size=(3,3,7), activation='relu',input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH, IMAGE_CHANNEL),padding='same')(input_layer)
    
    # output_layer_1_drop3d = layers.SpatialDropout3D(rate=0.3, data_format='channels_last')(output_layer_1_conv,training=True)
    
    # output_layer_2_conv = layers.Conv3D(filters=16, kernel_size=(3,3,5), activation='relu',padding='same')(output_layer_1_drop3d)
    
    # output_layer_2_drop3d = layers.SpatialDropout3D(rate=0.3, data_format='channels_last')(output_layer_2_conv,training=True)
    
    # output_layer_3_conv = layers.Conv3D(filters=32, kernel_size=(3,3,3), activation= 'relu')(output_layer_2_drop3d)
    
    # output_layer_3_reshaped = layers.Reshape((output_layer_3_conv.shape[1], output_layer_3_conv.shape[2], output_layer_3_conv.shape[3]*output_layer_3_conv.shape[4]))(output_layer_3_conv)
    
    # output_layer_4_conv = layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu')(output_layer_3_reshaped)
    
    # output_layer_4_flatten = layers.Flatten()(output_layer_4_conv)
    
    # output_layer_4_drop = layers.Dropout(rate=0.4)(output_layer_4_flatten,training=True)
    
    # output_layer_4_dense = layers.Dense(256, activation='relu')(output_layer_4_drop)
    
    # output_layer_5_conv = layers.Dropout(0.4)(output_layer_4_dense,training=True)
    
    # output_layer_5_dense = layers.Dense(128, activation='relu')(output_layer_5_conv)
    
    # model = Model(inputs=input_layer, outputs=output_layer_5_dense)

    
    input_layer = layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH, IMAGE_CHANNEL))

    output_layer_1_conv = layers.Conv3D(filters=8, kernel_size=(3,3,7), activation='relu',input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH, IMAGE_CHANNEL),padding='same')(input_layer)

    output_layer_1_drop3d = layers.SpatialDropout3D(rate=0.3, data_format='channels_last')(output_layer_1_conv,training=True)

    output_layer_2_conv = layers.Conv3D(filters=16, kernel_size=(3,3,5), activation='relu',padding='same')(output_layer_1_drop3d)

    output_layer_2_drop3d = layers.SpatialDropout3D(rate=0.3, data_format='channels_last')(output_layer_2_conv,training=True)

    output_layer_3_conv = layers.Conv3D(filters=32, kernel_size=(3,3,3), activation= 'relu')(output_layer_2_drop3d)

    output_layer_3_reshaped = layers.Reshape((output_layer_3_conv.shape[1], output_layer_3_conv.shape[2], output_layer_3_conv.shape[3]*output_layer_3_conv.shape[4]))(output_layer_3_conv)

    output_layer_4_conv = layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu')(output_layer_3_reshaped)

    output_layer_5_SA_dot = layers.Attention()([output_layer_4_conv, output_layer_4_conv])

    output_layer_5_softmax = layers.Softmax()(output_layer_5_SA_dot)

    output_layer_5_SA_concat = layers.Attention(score_mode='concat')([output_layer_5_softmax, output_layer_4_conv])

    output_layer_5_normalization = layers.LayerNormalization()(output_layer_5_SA_concat)

    output_layer_6_flatten = layers.Flatten()(output_layer_5_normalization)

    output_layer_6_drop = layers.Dropout(rate=0.4)(output_layer_6_flatten,training=True)

    output_layer_6_dense = layers.Dense(256, activation='relu')(output_layer_6_drop)

    output_layer_7_conv = layers.Dropout(0.4)(output_layer_6_dense,training=True)

    output_layer_7_dense = layers.Dense(128, activation='relu')(output_layer_7_conv)

    model = Model(inputs=input_layer, outputs=output_layer_7_dense)
    
    print(model.summary())
    return model
    
    
if(TEST_BLOCKS):
    model = createModel()
    print(model.summary())
    

### Prototypical Network

In [247]:
def calc_euclidian_dists(x, y):
    """
    calc_euclidian_dists: Calculates the euclidian distance between two tensors
    :param x: Tensor of shape (n, d)
    :param y: Tensor of shape (m, d)
    :return: Tensor of shape (n, m) with euclidian distances
    """
    n = x.shape[0]
    m = y.shape[0]
    x = tf.tile(tf.expand_dims(x, 1), [1, m, 1])
    y = tf.tile(tf.expand_dims(y, 0), [n, 1, 1])
    return tf.reduce_mean(tf.math.pow(x - y, 2), 2)   

In [248]:
class Prototypical(Model):
    def __init__(self, model, w, h, d, c):
        '''
        model: encoder model
        w: width of input image
        h: height of input image
        d: depth of input image
        c: number of channels of input image
        '''
        super(Prototypical, self).__init__()
        self.w, self.h, self.d, self.c = w, h, d, c
        self.encoder = model

    def call(self, support, query, support_labels, query_labels, K, C, N,n_times,training=True):
        '''
        support: support images (25, 11, 11, 30, 1)
        query: query images (75, 11, 11, 30, 1)
        supppor_labels: support labels (25, 5)
        query_labels: query labels (75, 5)
        K: number of support images per class
        C: number of classes
        N: number of query images per class
        n_times: number of times to pass the query images for variance calculation
        training: True if training, False if testing
        '''
        cat = tf.concat([support,query], axis=0)
        loss = 0
        all_predictions = []
        y = np.zeros((int(C*N),C))
        for i in range(int(C*N)) :
            x = support_labels.index(query_labels[i])
            y[i][x] = 1

            
        for i in range(n_times) :
            # Pass through encoder to get embeddings.
            z = self.encoder(cat)

            # Reshape embeddings to separate support and query embeddings.
            # For prototypes, we reshape C x K embeddings to C x K x D, ie. each class has K examples, each of D dimensions.
            z_support = tf.reshape(z[:C * K],[C, K, z.shape[-1]])
            # For query, we simply take the remaining embeddings.
            z_query = z[C * K:]

            # The prototypes are simply the mean of the support embeddings.
            z_prototypes = tf.math.reduce_mean(z_support, axis=1)

            # Take the euclidian distance between the query embeddings and the prototypes.
            distances = calc_euclidian_dists(z_query, z_prototypes)

            # Calculate the log softmax of the distances. These are the preictions for the current pass.
            predictions = tf.nn.log_softmax(-distances, axis=-1)

            # Calcyulate the loss for the current pass and add it to the total loss.
            loss += - tf.reduce_mean((tf.reduce_sum(tf.multiply(y, predictions), axis=-1)))

            # Append the predictions for the current pass to the list of predictions.
            all_predictions.append(predictions)
        
        
        if training:
            # Convert the list of predictions to a tensor.
            predictions = tf.convert_to_tensor(np.reshape(np.asarray(all_predictions),(n_times,int(C*N),C)))

            # Calculate the standard deviation of the predictions.
            std_predictions = tf.math.reduce_std(predictions,axis=0)

            # Calculate the standard deviation of the true labels.
            std = tf.reduce_sum(tf.reduce_sum(tf.multiply(std_predictions,y),axis=1))

            # Add the standard deviation to the loss.
            loss += MC_LOSS_WEIGHT*std

            # Calculate the mean prediction.
            mean_predictions = tf.reduce_mean(predictions,axis=0)

            # Calculate the accuracy for each query patch.
            # Check if the index of max probability is equal to the true class index.
            mean_eq = tf.cast(tf.equal( tf.cast(tf.argmax(mean_predictions, axis=-1), tf.int32), tf.cast(tf.argmax(y,axis=-1), tf.int32)), tf.float32)
            
            # Calculate the mean accuracy.
            mean_accuracy = tf.reduce_mean(mean_eq)
            
            return loss, mean_accuracy, mean_predictions

        else:
            # Calculate the mean predictions.
            mean_predictions = tf.reduce_mean(all_predictions,axis=0)
            
            # Get the index of the max probability for each query patch.
            mean_predictions_indices = tf.argmax(mean_predictions,axis=1)
            
            # Calculate the accuracy for each query patch.
            # Check if the index of max probability is equal to the true class index.
            mean_eq = tf.cast(tf.equal(tf.cast(mean_predictions_indices, tf.int32), tf.cast(tf.argmax(y, axis=-1), tf.int32)), tf.float32)
            
            # Calculate the mean accuracy.
            mean_accuracy = tf.reduce_mean(mean_eq)
            
            
            '''
            Explaination for classwise_mean_acc:
            
            What we need to get?
                We need to get the mean accuracy for each class.
            
            How to get it?
                We loop over all the query patches.
                x is the index of the true class of the current query patch.
                We check if the index of max probability is equal to the true class index.
                if yes, we append 1 to the list of correct predictions for the current class.
                if no, we append 0 to the list of correct predictions for the current class.
                
                After the loop, we calculate the mean accuracy for each class.
                To do this, we simply divide the number of correct predictions for each class by the total number of predictions for each class.
                    where sum(acc_list) represents the number of correct predictions for the current class.
                    and len(acc_list) represents the total number of predictions for the current class.
            
            And done we have the mean accuracy for each class.
            '''
            classwise_mean_acc = [[] for _ in range(tC)]
            std = 0
            for i in range(int(C * N)):
                #  Classwise mean accuracy
                x = support_labels.index(query_labels[i])
                
                is_correct = (mean_predictions_indices[i] == x)
                classwise_mean_acc[x].append(int(is_correct))
                
                #  ----------------------
                # Standard deviation
                
                # Get all the predictions for the current query patch from all the n_times.
                p_i = np.array([p[i,:] for p in all_predictions])
                
                # Get the standard deviation of the predictions for the current query patch.
                std += tf.math.reduce_std(p_i, axis=0)[x]

            # Calculate the mean accuracy for each class
            classwise_mean_acc = [sum(acc_list) / len(acc_list)
                                if acc_list else 0 for acc_list in classwise_mean_acc]
            
            loss += MC_LOSS_WEIGHT*std
            
            return loss, all_predictions, mean_accuracy, classwise_mean_acc, y


        def save(self, model_path):
            self.encoder.save_weights(model_path)

        def load(self, model_path):
            self.encoder(tf.zeros([1, self.w, self.h, self.c]))
            self.encoder.load_weights(model_path)

## Data Loaders for Model

### For Training

In [249]:
def createTrainingEpisode(patches:list, labels:list, K:int, C:int, N:int ):
    """
    createTrainingEpisode creates a training episode for the N-way K-shot learning task.
    
    :param patches: list of all patches classified into different classes.
    :param labels: list of classes from which the traning episode is to be created.
    :param K: number of patches per class in the support set.
    :param C: number of classes in the training episode.
    :param N: number of patches per class in the query set.
    :return queryPatches, queryLabels, supportPatches, supportLabels: training episode
    
    Algorithm:
    - Select N classes from the list of labels. They should be unique.
    - For each class, select K+Q patches. They should be unique.
        - First K patches are support patches.
        - Last Q patches are query patches.
        - Append the support patches to supportPatches.
        - Append the query patches to queryPatches.
        - Append the class label to queryLabels Q times.
    - Shuffle the queryPatches and queryLabels in the same order.
    - Convert the queryPatches and supportPatches to tensors.
    
    """
    
    selectedLabels = random.sample(labels, C)
    supportPatches = []
    supportLabels = list(selectedLabels)
    queryPatches = []
    queryLabels = []
    
    for n in selectedLabels:
        sran_indices = np.random.choice(len(patches[n-1]),K,replace=False)  # for class no X-1: select K samples 
        supportPatches.extend( patches[n-1][sran_indices,:,:,:,:])
        qran_indices = np.random.choice(len(patches[n-1]),N,replace=False)  # N Samples for Query
        queryPatches.extend(patches[n-1][qran_indices,:,:,:,:])
        queryLabels.extend([n]*N)
    
    shuffled = list(zip(queryPatches, queryLabels))
    random.shuffle(shuffled)
    queryPatches, queryLabels = zip(*shuffled)
    
    queryPatches = tf.convert_to_tensor(np.reshape(np.asarray(queryPatches),(C*N,IMAGE_HEIGHT,IMAGE_WIDTH,IMAGE_DEPTH,IMAGE_CHANNEL)),dtype=tf.float32)
    supportPatches = tf.convert_to_tensor(np.reshape(np.asarray(supportPatches),(C*K,IMAGE_HEIGHT,IMAGE_WIDTH,IMAGE_DEPTH,IMAGE_CHANNEL)),dtype=tf.float32)
    
    return queryPatches, queryLabels, supportPatches, supportLabels


In [250]:
def train_step(support, query, support_labels, query_labels, K, C, N):
    # Forward & update gradients
    with tf.GradientTape() as tape:
        loss, mean_accuracy, mean_predictions = ProtoModel(support, query, support_labels, query_labels, K, C, N,N_TIMES,training=True)
    gradients = tape.gradient(loss, model.trainable_variables)
    
    # A gradient simply measures the change in all weights with regard to the change in error. You can also think of a gradient as the slope of a function. The higher the gradient, the steeper the slope and the faster a model can learn. But if the slope is zero, the model stops learning
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # Log loss and accuracy for step
    train_loss(loss)
    train_acc(mean_accuracy)


In [251]:
@timeIt
def trainingEpochs(patches, labels, n_epochs, n_episodes):
    """
    trainingEpochs function trains the model for n_epochs and n_episodes.
    
    :param patches: image patches to be trained
    :param labels: corresponding labels to be used
    :param n_epochs: number of epochs
    :param n_episodes: number of episodes
    :return: None
    """
    
    template = 'Epoch {}/{}, Episode {}/{}, Train Loss: {:.2f}, Train Accuracy: {:.2f}'
    # for epoch in tqdm(range(n_epochs), desc='Epochs'):
    #     train_loss.reset_states()
    #     train_acc.reset_states()
    #     for episode in tqdm(range(n_episodes), desc=f'Episodes (Loss: {l:.2f}, Acc: {a:.2f})'):
    
    trainObj = tqdm(total=n_episodes * n_epochs, desc=f'Epoch 1/{n_epochs}, Episode 1/{n_episodes}')
    for epoch in range(n_epochs):
        train_loss.reset_states()
        train_acc.reset_states()
        for episode in range(n_episodes):
            queryPatches, queryLabels, supportPatches, supportLabels = createTrainingEpisode(patches, labels, TRAIN_K, TRAIN_C, TRAIN_N)
            train_step(supportPatches, queryPatches,supportLabels,  queryLabels, TRAIN_K, TRAIN_C, TRAIN_N)
            # clear_output(wait=True)
            trainObj.set_description(
                f'Epoch {epoch+1}/{n_epochs}, Episode {episode+1}/{n_episodes} (Loss: {train_loss.result().numpy()*100:.2f}, Acc: {train_acc.result().numpy()*100:.2f})')
            trainObj.update(1)
            if(VERBOSE):
                print(template.format(epoch+1, n_epochs, episode+1, n_episodes, train_loss.result()*100, train_acc.result()*100))
                trainingData.append([train_loss.result(),  train_acc.result()*100])
                plotData(trainingData)
        
        if(epoch and epoch % 5 == 0):
            checkpoint.save(file_prefix=checkpoint_prefix_train)    
    trainObj.close()
        

### For Tuning

In [252]:
def createTunningEpisodes(patches:list, labels:list, K:int, C:int, N:int):
    """
    createTuningEpisodes creates a tuning episode for the N-way K-shot learning task.
    
    :param patches: list of all patches classified into different classes.
    :param labels: list of classes from which the tuning episode is to be created.
    :param K: number of patches per class in the support set.
    :param C: number of classes in the tuning episode.
    :param N: number of patches per class in the query set.
    :return queryPatches, queryLabels, supportPatches, supportLabels: tuning episode
    
    Algorithm:
    - Select C classes from the list of labels. They should be unique.
    - For each selected class.
        - Shuffle the patches of that class.
        - First K patches are support patches.
        - Next N patches are query patches. 
        - Append the support patches to supportPatches.
        - Append the query patches to queryPatches.
        - Append the class label to queryLabels N times.
    - Shuffle the queryPatches and queryLabels in the same order.
    - Convert the queryPatches and supportPatches to tensors.
    
    """

    selected_classes = np.random.choice(labels,C,replace=False)
    supportLabels  = list(selected_classes)
    queryLabels = []
    supportPatches = []
    queryPatches = []
    
    for x in selected_classes :
        y = labels.index(x)
        np.random.shuffle(patches[y])    
        supportPatches.extend(patches[y][:K,:,:,:,:])  # 1st K patches for support set
        queryPatches.extend(patches[y][K:K+N,:,:,:,:])   # next N patches for query set
        queryLabels.extend([x]*N)            
          # next 5 labels for query set
    
    shuffled = list(zip(queryPatches, queryLabels))
    random.shuffle(shuffled)
    queryPatches, queryLabels = zip(*shuffled)
    
    queryPatches = tf.convert_to_tensor(np.reshape(np.asarray(queryPatches),(C*N,IMAGE_HEIGHT,IMAGE_WIDTH,IMAGE_DEPTH,IMAGE_CHANNEL)),dtype=tf.float32)
    supportPatches = tf.convert_to_tensor(np.reshape(np.asarray(supportPatches),(C*K,IMAGE_HEIGHT,IMAGE_WIDTH,IMAGE_DEPTH,IMAGE_CHANNEL)),dtype=tf.float32)
    
    return queryPatches, queryLabels, supportPatches, supportLabels
    

In [253]:
def tune_step(support, query, support_labels, query_labels, K, C, N):
    # Forward & update gradients
    with tf.GradientTape() as tape:
        loss, mean_accuracy, mean_predictions = ProtoModel(support, query, support_labels, query_labels, K, C, N,N_TIMES,training=True)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # Log loss and accuracy for step
    tune_loss(loss)
    tune_acc(mean_accuracy)

In [254]:
@timeIt
def tunningEpochs(patches, labels, n_epochs, n_episodes):
    """
    trainingEpochs function trains the model for n_epochs and n_episodes.
    
    :param patches: image patches to be trained
    :param labels: corresponding labels to be used
    :param n_epochs: number of epochs
    :param n_episodes: number of episodes
    :return: None
    """
    template = 'Epoch {}/{}, Tune Loss: {:.2f}, Tune Accuracy: {:.2f}'

    epochObj = tqdm(range(n_epochs), desc='Epochs')
    for epoch in epochObj: 
        tune_loss.reset_states()  
        tune_acc.reset_states()    
        for epi in range(n_episodes):  
            queryPatches, queryLabels, supportPatches, supportLabels = createTunningEpisodes(patches, labels, TUNE_K, TUNE_C, TUNE_N)    
            tune_step(supportPatches, queryPatches,supportLabels, queryLabels, TUNE_K, TUNE_C, TUNE_N)   
            # clear_output(wait=True)   
        epochObj.set_postfix(
            {'Loss': tune_acc.result().numpy()*100, 'Acc': tune_loss.result().numpy()}, refresh=True)
        if(VERBOSE):
            print(template.format(epoch+1, n_epochs,tune_loss.result(),tune_acc.result()*100))
            tunningData.append([tune_loss.result(),  tune_acc.result()*100])
            plotData(tunningData)
        if (epoch+1)%5 == 0 :
            checkpoint.save(file_prefix = checkpoint_prefix_tune) 
    epochObj.close()

### For Testing

In [255]:
def createTestingEpisode(patches, labels, K, C, i, f):
    selected_classes = labels[i:f]   # [1, 2, 3, 4, 5, 6, 7, 8]
    support_labels = list(selected_classes)
    query_labels = []
    support_patches = []
    query_patches = []
    for x in selected_classes :
        y = labels.index(x)
        
        support_imgs = patches[y][:K,:,:,:,:]
        query_imgs = patches[y][K:,:,:,:,:]
        support_patches.extend(support_imgs)
        query_patches.extend(query_imgs)
        for i in range(query_imgs.shape[0]) :
            query_labels.append(x)
    temp1 = list(zip(query_patches, query_labels)) 
    random.shuffle(temp1) 
    query_patches, query_labels = zip(*temp1)
    x = len(query_labels)
    query_patches = tf.convert_to_tensor(np.reshape(np.asarray(query_patches),(x,IMAGE_HEIGHT,IMAGE_WIDTH,IMAGE_DEPTH,IMAGE_CHANNEL)),dtype=tf.float32)
    support_patches = tf.convert_to_tensor(np.reshape(np.asarray(support_patches),(C*K,IMAGE_HEIGHT,IMAGE_WIDTH,IMAGE_DEPTH,IMAGE_CHANNEL)),dtype=tf.float32)
    return query_patches, support_patches, query_labels, support_labels,x   

In [256]:
def test_step(support, query, support_labels, query_labels, K, C, y):
    loss, mc_predictions, mean_accuracy, classwise_mean_acc, y = ProtoModel(support, query, support_labels, query_labels, K, C, y,N_TIMES,training=False)
    return loss, mc_predictions, mean_accuracy, classwise_mean_acc, y

In [257]:
@timeIt
def testingEpochs(patches, labels, n_epochs):
    """
    testingEpochs function tests the model for n_epochs.
    
    :param patches: image patches to be trained
    :param labels: corresponding labels to be used
    :param n_epochs: number of epochs
    :return: None
    """
    
    epochObj = tqdm(range(n_epochs), desc=f'Epochs')
    
    for epoch in epochObj:
        test_loss.reset_states()  
        test_acc.reset_states()     
        
        tquery_patches1, tsupport_patches1, query_labels1, support_labels1, x1 = createTestingEpisode(patches,labels,TEST_K,TEST_C,0,3)    
        loss1, mc_predictions1, mean_accuracy1, classwise_mean_acc1, y1 = test_step(tsupport_patches1, tquery_patches1,support_labels1, query_labels1, TEST_K, TEST_C, y=x1/3) 
        tquery_patches2, tsupport_patches2, query_labels2, support_labels2, x2 = createTestingEpisode(patches,labels,TEST_K,TEST_C,3,6)    
        loss2, mc_predictions2, mean_accuracy2, classwise_mean_acc2, y2 = test_step(tsupport_patches2, tquery_patches2,support_labels2, query_labels2, 5, 3, x2/3)
        
        oa1 = mean_accuracy1
        oa2 = mean_accuracy2
        epochObj.set_postfix(
            {'OA1': oa1.numpy(), 'OA2': oa2.numpy()}, refresh=True)
        if(VERBOSE):
            print("=========================================")
            print(f"Epoch {epoch+1}/{n_epochs}")
            print("-----------------------------------------")
            print(f"Overall Accuracy 1 (OA1): {mean_accuracy1}")
            # Class Wise Accuracy
            for i in range(TEST_C):
                print(f"Class {i+1} Accuracy: {classwise_mean_acc1[i]}")
            print(f"Loss: {loss1.numpy():.3f}")
            print("-----------------------------------------")
            print(f"Overall Accuracy 2 (OA2): {mean_accuracy2}")
            # Class Wise Accuracy
            for i in range(TEST_C):
                print(f"Class {i+1+TEST_C} Accuracy: {classwise_mean_acc2[i]}")
            print(f"Loss: {loss2.numpy():.3f}")
            print("=========================================")
            
            testingData.append([mean_accuracy1*100, mean_accuracy2*100, loss1.numpy(), loss2.numpy()])
            plotData(testingData, testing=True)
        # else:
            # clear_output(wait=True)
            # print("-----------------------------------------\n" + f"Epoch {epoch+1}/{n_epochs}\n" + "-----------------------------------------\n"
            # + f"Overall Accuracy 1 (OA1): {mean_accuracy1}\n" + f"Overall Accuracy 2 (OA2): {mean_accuracy2}")

    epochObj.close()
    return mc_predictions1, mc_predictions2, y1, y2

## Main

In [258]:
# Load Dataset
data = Data(DATASET, PCA_COMPONENTS, WINDOW_SIZE)

indian_pines


In [259]:
X, Y, patches = data.get_data()

In [260]:
NUM_CLASSES, TRAINING_CLASSES, TRAINING_LABELS, TUNNING_LABELS, TESTING_CLASSES, TESTING_LABELS, TRAINING_PATCHES,TUNNING_PATCHES, TESTING_PATCHES = data.load_defaults()

6


In [261]:
# Create instance of the model
model = createModel()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 11, 11, 30,  0           []                               
                                 1)]                                                              
                                                                                                  
 conv3d_24 (Conv3D)             (None, 11, 11, 30,   512         ['input_9[0][0]']                
                                8)                                                                
                                                                                                  
 spatial_dropout3d_16 (SpatialD  (None, 11, 11, 30,   0          ['conv3d_24[0][0]']              
 ropout3D)                      8)                                                          

In [262]:
# Create instance of the Prototypical Network
ProtoModel = Prototypical(model, IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_DEPTH, IMAGE_CHANNEL)

In [263]:
# Create instance of the Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

In [264]:
# Create instance of the Checkpoint
checkpoint = tf.train.Checkpoint(optimizer=optimizer, ProtoModel = ProtoModel)

In [265]:
# Train the model
trainingEpochs(patches, TRAINING_LABELS, TRAINING_EPOCH, TRAINING_EPISODE)

Epoch 3/3, Episode 5/5 (Loss: 159.98, Acc: 22.67): 100%|██████████| 15/15 [00:02<00:00,  7.13it/s, Loss=0, Acc=0]

Function 'trainingEpochs' executed in 2.1115s


In [266]:
tunningEpochs(TUNNING_PATCHES, TESTING_LABELS, TUNNING_EPOCH, TUNNING_EPISODE)

Epochs: 100%|██████████| 3/3 [00:01<00:00,  2.37it/s, Loss=38.3, Acc=1.08]

Function 'tunningEpochs' executed in 1.2705s


In [267]:
mc_predictions1, mc_predictions2, y1, y2 =  testingEpochs(TESTING_PATCHES, TESTING_LABELS, TESTING_EPOCH)

Epochs: 100%|██████████| 10/10 [00:17<00:00,  1.75s/it, OA1=0.223, OA2=0.399]

Function 'testingEpochs' executed in 17.4637s


In [268]:

from lib.Stats import Stats
stats = Stats(mc_predictions1, mc_predictions2, y1, y2)

In [269]:
stats.printReport()

20.00025933105554 Kappa accuracy (%)


31.21869782971619 Overall accuracy (%)


39.56349541415545 Average accuracy (%)




              precision    recall  f1-score   support

           0       0.15      0.44      0.22        41
           1       0.80      0.17      0.28       232
           2       0.06      0.35      0.11        23
           3       0.09      0.60      0.16        15
           4       0.71      0.36      0.48       200
           5       0.39      0.45      0.42        88

    accuracy                           0.31       599
   macro avg       0.37      0.40      0.28       599
weighted avg       0.62      0.31      0.35       599



[[18  4 19  0  0  0]
 [95 40 97  0  0  0]
 [ 9  6  8  0  0  0]
 [ 0  0  0  9  4  2]
 [ 0  0  0 68 72 60]
 [ 0  0  0 22 26 40]]


In [270]:
if(SAVE_REPORT):
    stats.saveReport(report_path)

In [271]:
if(SAVE_MODEL):
    ProtoModel.save(model_save_path)